<a href="https://colab.research.google.com/github/BayramGurbuz/diabetic-retinopathy-detection/blob/main/dr_pred_with_tta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

|------------------------------------------------------------------------------|

---



In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"bayramgurbuz7","key":"97754030434bc270b2d8f20c67ee6108"}'}

In [ ]:
!pip install -q kaggle
!mkdir -p ~/.kaggle  # .kaggle klasörünü oluştur
!cp kaggle.json ~/.kaggle/  # Kaggle JSON dosyasını uygun klasöre kopyala
!chmod 600 ~/.kaggle/kaggle.json  # Dosya izinlerini ayarla

In [ ]:
!kaggle competitions download -c aptos2019-blindness-detection

 99% 9.42G/9.51G [00:31<00:00, 559MB/s]
100% 9.51G/9.51G [00:31<00:00, 325MB/s]


In [ ]:
import zipfile
import os

# ZIP dosyasının yolu
zip_path = '/content/aptos2019-blindness-detection.zip'

# Çıkartılacak dizin
output_dir = '/content/data/'

# Dizini oluştur (eğer yoksa)
os.makedirs(output_dir, exist_ok=True)

# ZIP dosyasını çıkart
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(output_dir)

print(f'ZIP dosyası {output_dir} dizinine çıkartıldı.')



ZIP dosyası /content/data/ dizinine çıkartıldı.


Ortam Hazırlığı

In [ ]:
!pip install -q efficientnet
import os
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
import efficientnet.tfkeras as efn
import matplotlib.pyplot as plt
import albumentations as A
from tqdm import tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 1.2 MB/s eta 0:00:00


Veri Yolunu Tanımla

In [ ]:
IMG_SIZE = 300
DATA_DIR = "/content/data/train_images"  # görüntülerin olduğu klasör
CSV_PATH = "/content/data/train.csv"     # etiket dosyası

df = pd.read_csv(CSV_PATH)
df['id_code'] = df['id_code'].apply(lambda x: x + '.png')
df['image_path'] = df['id_code'].apply(lambda x: os.path.join(DATA_DIR, x))


Preprocessing Fonksiyonu(Resize + Ben’s preprocessing)

In [ ]:
def ben_preprocess(image):
    image_blur = cv2.addWeighted(image, 4, cv2.GaussianBlur(image, (0,0), 10), -4, 128)
    return image_blur

def load_and_preprocess(path):
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    img = ben_preprocess(img)
    img = img.astype(np.float32) / 255.0
    return img


Ağır Data Augmentation

In [ ]:
train_aug = A.Compose([
    A.Rotate(limit=30, p=0.5),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.2),
    A.RandomBrightnessContrast(p=0.5),
    A.RandomGamma(p=0.3),
    A.ElasticTransform(p=0.3),
    A.ShiftScaleRotate(p=0.4),
    A.CLAHE(p=0.2),
    A.Blur(blur_limit=3, p=0.3),
    A.Resize(IMG_SIZE, IMG_SIZE)
])

val_aug = A.Compose([
    A.Resize(IMG_SIZE, IMG_SIZE)
])


/usr/local/lib/python3.11/dist-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)


Dataset Hazırla


In [ ]:
def preprocess_image(path, label, augment=True):
    image = load_and_preprocess(path.decode())

    if augment:
        image = train_aug(image=image)['image']
    else:
        image = val_aug(image=image)['image']

    return image, label

def tf_wrapper(path, label, augment):
    image, label = tf.numpy_function(preprocess_image, [path, label, augment], [tf.float32, tf.float32])
    image.set_shape([IMG_SIZE, IMG_SIZE, 3])
    label.set_shape([])
    return image, label

def get_dataset(df, batch_size=32, augment=True, shuffle=True):
    paths = df['image_path'].values
    labels = df['diagnosis'].values.astype(np.float32)

    ds = tf.data.Dataset.from_tensor_slices((paths, labels))
    if shuffle:
        ds = ds.shuffle(1024)
    ds = ds.map(lambda x, y: tf_wrapper(x, y, augment), num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return ds

train_df, val_df = train_test_split(df, test_size=0.15, stratify=df['diagnosis'], random_state=42)
train_ds = get_dataset(train_df, augment=True)
val_ds = get_dataset(val_df, augment=False, shuffle=False)

 EfficientNetB5 Modeli (Regresyon)

In [ ]:
def build_model():
    base = efn.EfficientNetB5(
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
        weights='imagenet',
        include_top=False
    )
    base.trainable = False  # İlk eğitim için

    model = models.Sequential([
        base,
        layers.GlobalAveragePooling2D(),
        layers.Dropout(0.5),
        layers.Dense(1, activation='linear')  # Regresyon
    ])
    return model

model = build_model()
model.compile(optimizer=Adam(1e-4), loss=MeanSquaredError())
model.summary()


115515256/115515256 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnet-b5 (Functional)    │ (None, 10, 10, 2048)   │    28,513,520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │         2,049 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 28,515,569 (108.78 MB)

 Trainable params: 2,049 (8.00 KB)

 Non-trainable params: 28,513,520 (108.77 MB)

Eğitim

In [ ]:
callbacks = [
    ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.3, verbose=1),
    EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True, verbose=1),
    ModelCheckpoint("best_model.h5", monitor='val_loss', save_best_only=True, verbose=1)
]

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=25,
    callbacks=callbacks
)


regresyon Çıktılarını Sınıfa Dönüştür (reg → class)

In [ ]:
import numpy as np

def reg_to_class(preds):
    """
    Regresyon tahminlerini (float) 0-4 arası sınıf etiketlerine dönüştürür.
    """
    return np.clip(np.round(preds), 0, 4).astype(int)


Kappa Score Hesaplama (Eval için)

In [ ]:
import numpy as np
from sklearn.metrics import cohen_kappa_score

# 1. Regresyon tahminlerini al
y_val_reg = model.predict(val_ds).flatten()

# 2. Tahminleri 0-4 arası sınıf etiketlerine dönüştür
y_val_pred = np.clip(np.round(y_val_reg), 0, 4).astype(int)

# 3. Gerçek etiketleri topla
y_val_true = np.concatenate([y for x, y in val_ds], axis=0).astype(int)

# 4. Kappa skoru hesapla
kappa = cohen_kappa_score(y_val_true, y_val_pred, weights='quadratic')
print("Quadratic Weighted Kappa Score:", kappa)


18/18 ━━━━━━━━━━━━━━━━━━━━ 32s 1s/step
Quadratic Weighted Kappa Score: 0.7719066175681957


🔧 Fine-Tuning Adımı: EfficientNetB5 Son Katmanlarını Açıp Yeniden Eğitme

In [ ]:
from tensorflow.keras.models import load_model

# En iyi modeli yükle (ilk eğitimi tamamladıktan sonra kaydetmiştik)
model = load_model("best_model.h5", compile=False)


EfficientNetB5'in Son Katmanlarını Aç

In [ ]:
# EfficientNetB5 gövdesini al
backbone = model.layers[0]
print(f"Toplam katman sayısı: {len(backbone.layers)}")

# Son N katmanı eğitilebilir yapalım (örneğin son 30)
for layer in backbone.layers[-30:]:
    layer.trainable = True


Toplam katman sayısı: 569


Modeli Yeniden Derle

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError

model.compile(
    optimizer=Adam(1e-5),
    loss=MeanSquaredError()
)


Callback’leri Tanımla ve Eğitimi Başlat

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

checkpoint_cb = ModelCheckpoint("finetuned_model.keras", monitor='val_loss', save_best_only=True, verbose=1)
earlystop_cb = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1)
reduce_lr_cb = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=2, verbose=1)

callbacks = [checkpoint_cb, earlystop_cb, reduce_lr_cb]


history_finetune = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=15,
    callbacks=callbacks
)


Epoch 1/15
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - loss: 0.8407   
Epoch 1: val_loss improved from inf to 0.58040, saving model to finetuned_model.keras
98/98 ━━━━━━━━━━━━━━━━━━━━ 183s 1s/step - loss: 0.8407 - val_loss: 0.5804 - learning_rate: 1.0000e-05
Epoch 2/15
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 799ms/step - loss: 0.7915
Epoch 2: val_loss improved from 0.58040 to 0.56142, saving model to finetuned_model.keras
98/98 ━━━━━━━━━━━━━━━━━━━━ 94s 953ms/step - loss: 0.7912 - val_loss: 0.5614 - learning_rate: 1.0000e-05
Epoch 3/15
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 798ms/step - loss: 0.7479
Epoch 3: val_loss improved from 0.56142 to 0.55806, saving model to finetuned_model.keras
98/98 ━━━━━━━━━━━━━━━━━━━━ 94s 953ms/step - loss: 0.7479 - val_loss: 0.5581 - learning_rate: 1.0000e-05
Epoch 4/15
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 802ms/step - loss: 0.7719
Epoch 4: val_loss improved from 0.55806 to 0.54173, saving model to finetuned_model.keras
98/98 ━━━━━━━━━━━━━━━━━━━━ 94s 954ms/step - loss: 0.7715 - val_los

Yeni Kappa Skorunu Hesapla

In [ ]:
from sklearn.metrics import cohen_kappa_score

# Regresyon çıktıları
y_val_reg = model.predict(val_ds).flatten()

# Sınıfa dönüştür
y_val_pred = np.clip(np.round(y_val_reg), 0, 4).astype(int)

# Gerçek etiketler
y_val_true = np.concatenate([y for x, y in val_ds], axis=0).astype(int)

# Kappa skorunu hesapla
kappa_finetuned = cohen_kappa_score(y_val_true, y_val_pred, weights='quadratic')
print("Fine-tuned Kappa Score:", kappa_finetuned)


18/18 ━━━━━━━━━━━━━━━━━━━━ 34s 1s/step
Fine-tuned Kappa Score: 0.8372109603365099


 Test-Time Augmentation (TTA)

In [ ]:
from tensorflow.keras.models import load_model
model = load_model("finetuned_model.keras")

TTA'da Kullanılacak Augmentasyonları Tanımlayalım

In [ ]:
# TTA için kullanılacak augmentations
tta_transforms = [
    A.HorizontalFlip(p=1.0),
    A.VerticalFlip(p=1.0),
    A.RandomRotate90(p=1.0),
    A.Transpose(p=1.0),
    A.RandomBrightnessContrast(p=1.0),
    A.NoOp(p=1.0)  # Orijinal görüntü
]


Bir görüntüye birden fazla transform uygulayıp tahmin alalım

In [ ]:
def tta_predict_single_image(model, image_path, tta_transforms):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))

    preds = []
    for transform in tta_transforms:
        augmented = transform(image=image)['image']
        augmented = ben_preprocess(augmented)
        augmented = augmented.astype(np.float32) / 255.0
        augmented = np.expand_dims(augmented, axis=0)  # (1, H, W, C)
        pred = model.predict(augmented)[0][0]  # Regresyon sonucu
        preds.append(pred)

    return np.mean(preds)  # Ortalamasını al


TTA ile tüm validation seti üzerinde tahmin alalım

In [ ]:
tta_preds = []
y_true = []

for i in tqdm(range(len(val_df))):
    path = val_df.iloc[i]['image_path']
    label = val_df.iloc[i]['diagnosis']

    pred = tta_predict_single_image(model, path, tta_transforms)
    tta_preds.append(pred)
    y_true.append(label)


  0%|          | 0/550 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 12s 12s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step


  0%|          | 1/550 [00:12<1:54:07, 12.47s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step


  0%|          | 2/550 [00:13<50:27,  5.52s/it]  

...

100%|█████████▉| 548/550 [06:32<00:01,  1.46it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step


100%|█████████▉| 549/550 [06:33<00:00,  1.48it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step


100%|██████████| 550/550 [06:34<00:00,  1.39it/s]


Kappa Score Hesapla

In [ ]:
# Regresyon değerlerini sınıfa çevir
tta_classes = np.clip(np.round(tta_preds), 0, 4).astype(int)
y_true = np.array(y_true).astype(int)

from sklearn.metrics import cohen_kappa_score
tta_kappa = cohen_kappa_score(y_true, tta_classes, weights='quadratic')
print("TTA ile Quadratic Weighted Kappa Score:", tta_kappa)


TTA ile Quadratic Weighted Kappa Score: 0.8320799388019622


modeli test setine uygula

Test Verisini Yükle

In [ ]:
import pandas as pd
import os

TEST_DIR = '/content/data/test_images'
TEST_CSV = '/content/data/test.csv'

test_df = pd.read_csv(TEST_CSV)
test_df['image_path'] = test_df['id_code'].apply(lambda x: os.path.join(TEST_DIR, x + '.png'))

print("Toplam test görüntüsü:", len(test_df))

Toplam test görüntüsü: 1928


Test Dataset Oluştur (Augmentations ile TTA için)

In [ ]:
tta_transforms = [
    A.Compose([A.HorizontalFlip(p=1.0), A.Resize(IMG_SIZE, IMG_SIZE)]),
    A.Compose([A.VerticalFlip(p=1.0), A.Resize(IMG_SIZE, IMG_SIZE)]),
    A.Compose([A.Rotate(limit=90, p=1.0), A.Resize(IMG_SIZE, IMG_SIZE)]),
    A.Compose([A.NoOp(), A.Resize(IMG_SIZE, IMG_SIZE)])
]

def load_test_image(path):
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = ben_preprocess(image)
    return image.astype(np.float32) / 255.0

def get_tta_predictions(model, paths, transforms):
    predictions = []
    for path in tqdm(paths, desc="TTA prediction"):
        img = load_test_image(path)
        preds = []
        for t in transforms:
            aug_img = t(image=img)['image']
            aug_img = tf.convert_to_tensor(aug_img)
            aug_img = tf.image.resize(aug_img, (IMG_SIZE, IMG_SIZE))
            aug_img = tf.expand_dims(aug_img, 0)  # (1, H, W, C)
            pred = model.predict(aug_img, verbose=0)[0][0]  # regress value
            preds.append(pred)
        predictions.append(np.mean(preds))  # TTA ortalaması
    return np.array(predictions)


modeli yükle

In [ ]:
from tensorflow.keras.models import load_model

model = load_model('finetuned_model.keras', compile=False)


tahmin Yap ve Sınıfa Dönüştür

In [ ]:
test_paths = test_df['image_path'].values
test_preds_reg = get_tta_predictions(model, test_paths, tta_transforms)
test_preds_cls = np.clip(np.round(test_preds_reg), 0, 4).astype(int)


TTA prediction: 100%|██████████| 1928/1928 [14:00<00:00,  2.29it/s]


Submission Dosyasını Hazırla

In [ ]:
submission_df = pd.DataFrame({
    'id_code': test_df['id_code'],
    'diagnosis': test_preds_cls
})
submission_df.to_csv('submission.csv', index=False)
print("submission.csv dosyası oluşturuldu.")


submission.csv dosyası oluşturuldu.


streamlit yayınlama

In [ ]:
!pip install streamlit

In [ ]:
!pip install streamlit pyngrok --quiet

In [ ]:
!ngrok config add-authtoken 30xh8FWh0zYXn2opXdzzkxM5CR9_2APib58oL5P6R18vfW9vT

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from pyngrok import ngrok
import time

# Mevcut ngrok tünellerini kontrol et ve kapat
try:
    tunnels = ngrok.get_tunnels()
    if tunnels:
        print("Mevcut ngrok tünelleri kapatılıyor...")
        for tunnel in tunnels:
            ngrok.disconnect(tunnel.public_url)
        time.sleep(2) # Tünellerin kapanması için biraz bekle

except Exception as e:
    print(f"Mevcut tünelleri kontrol ederken hata oluştu: {e}")


ngrok.kill()  # Mevcut bağlantıları temizle


# Streamlit için HTTP bağlantısını oluştur
# pyngrok'un varsayılan portu 8501 olduğu için belirtmeye gerek yok
# public_url = ngrok.connect("http://localhost:8501")

# ngrok.connect varsayılan olarak 80 portunu kullanır, Streamlit 8501 kullanır.
# Bu nedenle portu belirtmeliyiz.
try:
    public_url = ngrok.connect(addr="8501", proto="http")
    print(f"Streamlit uygulaman buradan erişilebilir: {public_url}")

    # Streamlit uygulamasını arka planda başlat
    # &> streamlit.log çıktıyı streamlit.log dosyasına yönlendirir
    # & komutu arka planda çalışmasını sağlar
    !streamlit run app.py &> streamlit.log &

except Exception as e:
    print(f"ngrok bağlantısı kurulurken hata oluştu: {e}")
    print("Lütfen ngrok token'ınızın doğru olduğundan ve başka bir ngrok oturumunun aktif olmadığından emin olun.")

Mevcut ngrok tünelleri kapatılıyor...


Streamlit uygulaman buradan erişilebilir: NgrokTunnel: "https://f34e8ba71a3c.ngrok-free.app" -> "http://localhost:8501"
